# Practice assignment: Advanced ensembling techniques
This assignment is graded by your `submission.json`.

The cell below creates a valid `submission.json` file, fill your answers in there. 

You can press "Submit Assignment" at any time to submit partial progress.

In [1]:
%%file submission.json
{
    "q1": 0,
    "q2": 843300,
    "q3": 1400,
    "q4": 3349.74057,
    "q5": 10329.20768,
    "q6": 8890.23464,
    "q7": 8692.77417,
    "q8": "data_channel_is_bus",
    "q9": 8451.2859,
    "q10": 8421.6219,
    "q11": "self_reference_min_shares",
    "q12": 8527.92717,
    "q13": 8485.01086,
    "q14": 8465.44037,
    "q15": "kw_avg_avg",
    "q16": 8426.97894,
    "q17": 2,
    "q18": 2880.08592,
    "q19": 0.84346,
    "q20": 8438.77113,
    "q21": 8411.04015,
    "q22": 8441.65406
}

Overwriting submission.json


In this programming assignment, you are going to work with a dataset based on the following data:

https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity

_Citation:_

* _K. Fernandes, P. Vinagre and P. Cortez. A Proactive Intelligent Decision Support System for Predicting the Popularity of Online News. Proceedings of the 17th EPIA 2015 - Portuguese Conference on Artificial Intelligence, September, Coimbra, Portugal._

The dataset contains the information about the internet news articles. In this assignment, you are going to predict a number of shares of the news article (target column: `shares`). The information about the features is available through the link above. You are going to construct several machine learning algorithms (XGBoost, LightGBM, CatBoost and Lasso) and blend them into the final ensemble.

In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('data.csv')

## 1

**q1:** How many missing values are there in the data? Provide the number of cells in the dataframe that contain NaNs.

In [3]:
# your code here
# your code here
nans = df.isna().sum().sum()
print(nans)

0


## 2

**q2:** What is the maximum number of shares among all the news articles presented in the data?

In [4]:
# your code here
maxn = df.shares.max()
print(maxn)

843300


## 3

**q3:** What is the median number of shares for the articles published on Monday?

In [5]:
# your code here
median_monday = df[df.weekday_is_monday == 1].shares.median()
print(median_monday)

1400.0


## 4

First, we separate the target from the dataframe with features (`df` -> `X`, `y`).

Next, let's split the data into train/val/test sets in the ratio 60:20:20. The idea is that we will use train set to train our models, val set to validate them and test set to calculate the final error of the blend. So, test set will be a completely unseen data.

To do this, use a regular `train_test_split` from `sklearn` to split `X` and `y` into train and val/test parts in the ratio 60:40. Then use `train_test_split` again, but to split the obtain val/test part into validation and test in the ratio 50:50. In each `train_test_split` application, use `random_state=13` and other default parameter values.

In the end, you should obtain `X_train`, `X_val`, `X_test` with the following shapes, respectively: (23786, 58), (7929, 58), (7929, 58). The same logic is with `y_train`, `y_val`, `y_test`.

**q4:** What is the mean value of target in the test part (`X_test`)? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [6]:
X = df.drop('shares', axis=1)
y = df['shares']

In [7]:
# your code here
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, random_state=13, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, random_state=13, test_size=0.5)

mean_tt = round(y_test.mean(), 5)
print(mean_tt)

3349.74057


## 5

Now let's train our first model - XGBoost. A link to the documentation: https://xgboost.readthedocs.io/en/latest/

We will use Scikit-Learn Wrapper interface for XGBoost (and the same logic applies to the following LightGBM and CatBoost models). Here, we work on the regression task - hence we will use `XGBRegressor`. Read about the parameters of `XGBRegressor`: https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor

The main list of XGBoost parameters: https://xgboost.readthedocs.io/en/latest/parameter.html# Look through this list so that you understand which parameters are presented in the library.

Take `XGBRegressor` with MSE objective (`objective='reg:squarederror'`), 200 trees (`n_estimators=200`), `learning_rate=0.01`, `max_depth=5`, `random_state=13` and all other default parameter values. Train it on the train set (`fit` function). 

**q5:** Calculate Root Mean Squared Error (RMSE) on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [8]:
# your code here
model = XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.01, objective="reg:squarederror", random_state=13)
model.fit(X_train, y_train)
y_pred_val = model.predict(X_val)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

print(round(rmse_val, 5))

10329.20768


## 6

In the task 5, we have decided to build 200 trees in our model. However, it is hard to understand whether it is a good decision - maybe it is too much? Maybe 150 is a better number? Or 100? Or 50 is enough?

During the training process, it is possible to stop constructing the ensemble if we see that the validation error does not decrease anymore. Using the same XGBoost model, call `fit` function (to train it) with `eval_set=[(X_val, y_val)]` (to evaluate the boosting model after building a new tree) and `early_stopping_rounds=50` (and other default parameter values). This `early_stopping_rounds` says that if the validation metric does not increase on 50 consequent iterations, the training stops.

**q6:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [9]:
# your code here
model = XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.01, objective="reg:squarederror", random_state=13)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)
y_pred_val = model.predict(X_val)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

print(round(rmse_val, 5))

8890.23464


## 7

Notes on parameter tuning: https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html

Here, we tuned some parameters of the algorithm. Take `XGBRegressor` with the following parameters:

* `objective='reg:squarederror'`
* `n_estimators=5000`
* `learning_rate=0.001`
* `max_depth=4`
* `gamma=1`
* `subsample=0.5`
* `random_state=13`
* all other default parameter values

Train it in the same manner, as in the task 6, but with `early_stopping_rounds=500`. 

**q7:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice the speed of the algorithm.

In [8]:
# your code here
xgb_model = XGBRegressor(n_estimators=5000, max_depth=4, learning_rate=0.001, 
                         objective="reg:squarederror", gamma=1, subsample=0.5, random_state=13)
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=500, verbose=False)
y_pred_val = xgb_model.predict(X_val)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

print(round(rmse_val, 5))

8692.77417


## 8

Calculate feature importances according to the model, trained in the task 7. 

**q8:** What is the name of the most important feature? Provide it as the answer. Do you understand why it might be important for the model?

Notice that by default, `XGBRegressor` calculates feature importance considering gain (`importance_type` parameter).

In [9]:
# your code here
feature_importances = dict(zip(X_train.columns, xgb_model.feature_importances_))
max_fi = max(feature_importances, key=feature_importances.get)

print(max_fi)

data_channel_is_bus


## 9

Let's move to LightGBM. We will work with `LGBMRegressor`.

LGBMRegressor parameters: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html#lightgbm.LGBMRegressor

The main list of LightGBM parameters: https://lightgbm.readthedocs.io/en/latest/Parameters.html Look through this list so that you understand which parameters are presented in the library.

Take `LGBMRegressor` with the following parameters, similar to the previous `XGBoost` model:

* `objective='regression'`
* `n_estimators=200`
* `learning_rate=0.01`
* `max_depth=5`
* `random_state=13`
* other default parameter values

Train it on the training data with `eval_set=[(X_val, y_val)]`, `eval_metric='rmse'`, `early_stopping_rounds=50` and all other default parameter values. 

**q9:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice the speed of the algorithm and compare it to the speed of XGBoost model.

In [10]:
# your code here
model = LGBMRegressor(max_depth=5, learning_rate=0.01, n_estimators=200, objective="regression", random_state=13)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric="rmse", early_stopping_rounds=50, verbose=False)
y_pred_val = model.predict(X_val)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

print(round(rmse_val, 5))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
8451.2859


## 10

Notes on parameter tuning: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

Here, we tuned some parameters of the algorithm. Take `LGBMRegressor` with the following parameters:

* `objective='regression'`
* `n_estimators=5000`
* `learning_rate=0.001`
* `max_depth=3`
* `lambda_l2=1.0`
* `boosting_type='goss'`
* `random_state=13`
* all other default parameter values

Train it in the same manner, as in the task 9, but with `early_stopping_rounds=500`. 

**q10:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [11]:
# your code here
model = LGBMRegressor(boosting_type="goss", max_depth=3, learning_rate=0.001, n_estimators=5000, 
                      objective="regression", random_state=13, lambda_l2=1.0)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric="rmse", early_stopping_rounds=500, verbose=False)
y_pred_val = model.predict(X_val)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

print(round(rmse_val, 5))

[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
8421.39852


## 11

Calculate feature importances according to the model, trained in the task 10. 

**q11:** What is the name of the most important feature? Provide it as the answer. 

Do you understand why it might be important for the model?

Notice that by default, `LGBMRegressor` calculates feature importance considering number of times the feature is used in the model (`importance_type` parameter).

In [12]:
# your code here
feature_importances = dict(zip(X_train.columns, model.feature_importances_))
max_fi = max(feature_importances, key=feature_importances.get)
print(max_fi)

self_reference_min_shares


## 12

Since some features are not important for the model, we can drop them in order to try to construct a better model which does not consider them at all.

Obtain new train and validation sets without the features with LightGBM importance less than 10 (the importances were computed in the task 11). Train the same model as in the task 10 on the new train set in the same manner. 

**q12:** Calculate RMSE on the new validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice that the new versions of train and validation sets are used only in this task and in blending.

In [13]:
# your code here
new_features = filter(lambda x: feature_importances[x] >= 10, feature_importances)
X_train_new = X_train[new_features]
X_val_new = X_val[X_train_new.columns]

lgb_model = LGBMRegressor(boosting_type="goss", max_depth=3, learning_rate=0.001, 
                          n_estimators=5000, objective="regression", random_state=13, lambda_l2=1.0)
lgb_model.fit(X_train_new, y_train, eval_set=[(X_val_new, y_val)], eval_metric="rmse", 
              early_stopping_rounds=500, verbose=False)
y_pred_val = lgb_model.predict(X_val_new)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

print(round(rmse_val, 5))

[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
8422.73009


## 13

Let's move to CatBoost. We will work with `CatBoostRegressor`.

Info about `CatBoostRegressor`: https://catboost.ai/docs/concepts/python-reference_catboostregressor.html

CatBoost parameters: https://catboost.ai/docs/concepts/python-reference_parameters-list.html Look through this list so that you understand which parameters are presented in the library.

Take `CatBoostRegressor` with the following parameters, similar to the previous models:

* `loss_function='RMSE'`
* `iterations=200`
* `learning_rate=0.01`
* `max_depth=5`
* `random_state=13`
* other default parameter values

Train it on the training data with `eval_set=[(X_val, y_val)]`, `early_stopping_rounds=50` and all other default parameter values. 

**q13:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Notice the speed of the algorithm and compare it to the speed of XGBoost and LightGBM models.

In [14]:
# your code here
model = CatBoostRegressor(iterations=200, learning_rate=0.01, loss_function="RMSE", max_depth=5, random_state=13)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)
y_pred_val = model.predict(X_val)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

print(round(rmse_val, 5))

8485.01086


## 14

Notes on parameter tuning: https://catboost.ai/docs/concepts/parameter-tuning.html

Here, we tuned some parameters of the algorithm. Take `CatBoostRegressor` with the following parameters:

* `loss_function='RMSE'`
* `n_estimators=5000`
* `learning_rate=0.001`
* `max_depth=9`
* `random_state=13`
* all other default parameter values

Train it in the same manner, as in the task 13, but with `early_stopping_rounds=500`. 

**q14:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [15]:
# your code here
cb_model = CatBoostRegressor(n_estimators=5000, learning_rate=0.001, loss_function="RMSE", max_depth=9, random_state=13)
cb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=500, verbose=False)
y_pred_val = cb_model.predict(X_val)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

print(round(rmse_val, 5))

8465.44037


## 15

Calculate feature importances according to the model, trained in the task 14. 

**q15:** What is the name of the most important feature? Provide it as the answer. 

Do you understand why it might be important for the model?

Notice that in case of regression, `CatBoostRegressor` calculates feature importance considering PredictionValuesChange: https://catboost.ai/docs/concepts/fstr.html#fstr__regular-feature-importance

In [16]:
# your code here
feature_importances = dict(zip(X_train.columns, cb_model.feature_importances_))
max_fi = max(feature_importances, key=feature_importances.get)
print(max_fi)

kw_avg_avg


## 16

Finally, take a `Lasso` model from `sklearn` with `alpha=10.0`, `random_state=13` and all other default parameter values. Train it on the train set. 

**q16:** Calculate RMSE on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [17]:
# your code here
lasso_model = Lasso(alpha=10.0, random_state=13)
lasso_model.fit(X_train, y_train)
y_pred_val = lasso_model.predict(X_val)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

print(round(rmse_val, 5))

8426.97894


## 17

Compare the results on the validation set of the trained models:

* XGBoost (task 7)
* LightGBM (task 12)
* CatBoost (task 14)
* Lasso (task 16)

**q17:** Which model has the best RMSE value on the validation set? For the answer, provide the following:

* 1 (if XGBoost was the best)
* 2 (if LightGBM was the best)
* 3 (if CatBoost was the best)
* 4 (if Lasso was the best)

In [ ]:
print(2)

## 18

Finally, let's move to blending the models that we obtained. First, calculate the predictions for the trained models on the validation set. Remember that LightGBM model used slightly different set of columns in the data.

After getting the predictions for the validation set, concatenate them into a single dataframe `X_val_blend`. The dataframe should look like this:

||xgb|lgb|cb|lasso|
|-|-|-|-|-|
|0|2298.947754|3728.088336|3680.924182|4270.039931|
|1|3208.189209|5243.744431|4487.549790|6755.853939|
|...|...|...|...|...|

Here, `xgb` column represents XGBoost predictions, `lgb` - LightGBM predictions, `cb` - CatBoost predictions, `lasso` - lasso predictions.

**q18:** For the answer, calculate the mean value of all model predictions in the last row of this column (`X_val_blend.iloc[-1]`). Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [18]:
# your code here
X_val_blend = pd.concat([pd.Series(xgb_model.predict(X_val), name="xgb"),
                         pd.Series(lgb_model.predict(X_val_new), name="lgb"),
                         pd.Series(cb_model.predict(X_val), name="cb"),
                         pd.Series(lasso_model.predict(X_val), name="lasso")],
                        axis=1)

print(round(X_val_blend.iloc[-1].mean(), 5))

2881.74948


## 19

Obtain a matrix of pairwise Pearson Correlation Coefficient (PCC) values for the column of the dataframe `X_val_blend`. Find a pair of model predictions with the highest PCC value (don't consider 1.0 values of correlations with themselves). 

**q19:** What is this value equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [19]:
# your code here
corr_matrix = X_val_blend.corr()
max_corr = corr_matrix[corr_matrix < 1.0].max().max()

print(round(max_corr, 5))

0.84455


## 20

Blend models into the ensemble with the weights 0.25, 0.25, 0.25 and 0.25 (just mean value of the predictions). 

**q20:** Calculate RMSE of such ensemble on the validation set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Compare it with RMSE of each model and think whether this is a good ensemble.

In [20]:
# your code here
y_pred_val = X_val_blend.mean(axis=1)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

print(round(rmse_val, 5))

8439.26508


## 21

Tune the weights of the ensemble. Run each model weight through `np.linspace(0, 1, 101)`, so that all possible values of each weight will be [0.0, 0.01, 0.02, ..., 0.99, 1.0]. Skip each combinations of weights, if their sum is not equal to 1.0. If the sum of the weights in the combination is equal to 1.0, though, get ensemble prediction on the validation set using these weights and calculate RMSE value.

In the end, select a combination of weights with the best RMSE value - these are the best weights for the ensemble. 

**q21:** What is their corresponding RMSE value equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

Compare RMSE value of the ensemble with RMSE values of the models in it. Is the ensemble better?

_Hint. You probably want to save RMSE with the corresponding weights for each valid combination into some array. Also this weight tuning might be implemented as quadriple nested loop, or you may think about other ways of implementing it. You can track tuning progress using `tqdm` module._

In [21]:
# your code here
from itertools import combinations_with_replacement
from tqdm import tqdm

best_weights = None
best_rmse_val = float("inf")

possible_weights = combinations_with_replacement(np.linspace(0, 1, 101), 4)
possible_weights = filter(lambda x: sum(x) == 1.0, possible_weights)
possible_weights = np.array(list(possible_weights))

y_pred_val = X_val_blend.to_numpy() @ possible_weights.T

for i in range(y_pred_val.shape[1]):
    rmse_val = mean_squared_error(y_val, y_pred_val[:, i], squared=False)
    if rmse_val < best_rmse_val:
        best_rmse_val = rmse_val
        best_weights = possible_weights[i]

print(round(best_rmse_val, 5))

8411.46345


## 22

Using the best weights obtained in the task 21, run the best ensemble on the test set. To do this, obtain model predictions on the test set (you can write them to the similar table to the one for the validation set in the task 18). Remember that LightGBM model uses slightly different set of columns.

**q22:** Calculate RMSE of the final ensemble on the test set. What is it equal to? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [22]:
# your code here
X_test_blend = pd.concat([pd.Series(xgb_model.predict(X_test), name="xgb"),
                          pd.Series(lgb_model.predict(X_test[X_train_new.columns]), name="lgb"),
                          pd.Series(cb_model.predict(X_test), name="cb"),
                          pd.Series(lasso_model.predict(X_test), name="lasso")],
                         axis=1)

y_pred_test = X_test_blend.apply(lambda x: np.average(x, weights=best_weights), axis=1)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)

print(round(rmse_test, 5))

8441.68222
